# Discrete Kalman Filter

In [96]:
import numpy as np
import pandas as pd
from scipy.linalg import expm
import pyqtgraph as pg
from pyqtgraph.Qt import QtCore, QtGui
%gui qt5
import matplotlib.pyplot as plt

### Plotting

In [64]:
#set up plottig GUI
app = QtGui.QApplication([])
pg.setConfigOption('background','w')   

In [84]:
win = pg.GraphicsWindow(title="Occupancy Detection GUI")
plot1 = win.addPlot()
plot1.setXRange(-6,6)
plot1.setYRange(0,6)
plot1.setLabel('left',text = 'Y position (m)')
plot1.setLabel('bottom', text= 'X position (m)')
s1 = plot1.plot([],[],pen=None,symbol='o')

### Helper Functions

In [2]:
def predictStep(x, A):
    xPred = np.matmul(A,x)
    return xPred

In [3]:
def innovationStep(xPred,z, H):
    innovation = np.subtract(z, np.matmul(H, xPred)) #innovation
    return innovation

In [4]:
def UpdateStep(xPred, nu, K):
    xNew = np.add(xPred, np.matmul(K, innovation))
    return xNew

In [53]:
def dataAssociation(rThetaMeasurement, filteredCentroidPolarFrames):
    #predict
    previousCentroids = filteredCentroidPolarFrames[-1]
    xPredAll = np.array([])
    for centroidIndex in list(previousCentroids['CentroidNumber']):
        xPrev = np.expand_dims(previousCentroids.loc[centroidIndex][:-1].values, axis=1)
        xPred = predictStep(xPrev, A)
        if len(xPredAll) == 0:
            xPredAll = xPred
        else:
            xPredAll = np.hstack((xPredAll, xPred))

    xPredAll = np.transpose(xPredAll) #keep consistent with other dataframes
    xPredDf = pd.DataFrame(xPredAll)
    xPredDf.columns = ['PredictedRange', 'PredictedDoppler', 'PredictedTheta', 'PredictedAngularVelocity']
    xPredDf['CentroidNumber'] = pd.Series(np.arange(xPredDf.shape[0]))

    #data associate
    distanceMatrix = np.eye(xPredDf.shape[0], rThetaMeasurement.shape[0])
    #create distance matrix
    for centroidIndex in xPredDf['CentroidNumber']:
        #access prev centroid information
        prevCentroidInfo = xPredDf.loc[centroidIndex]
        #compare against measurement centroid stats and find distances
        for centroidNumber in rThetaMeasurement['CentroidNumber']:
            #access measured centroid information
            measurementCentroidInfo = rThetaMeasurement.loc[centroidNumber]
            #find distances between measured and predicted centroids
            length = np.sqrt((prevCentroidInfo.PredictedRange**2) + (measurementCentroidInfo.R ** 2) - \
            (2*measurementCentroidInfo.R*prevCentroidInfo.PredictedRange*np.cos((prevCentroidInfo.PredictedTheta - measurementCentroidInfo.Theta))))
            #fill in distance matrix
            distanceMatrix[centroidIndex, centroidNumber] = length
    #global association
    correctAssociationMatrix = np.array([])
    #loop as many times as needed
    for loopIterator in range(0, min(max(xPredDf['CentroidNumber']),max(rThetaMeasurement['CentroidNumber']))+1) :
        #find minimum distance 
        predictedCentroidNumber, measuredCentroidNumber = (np.where(distanceMatrix == np.min(distanceMatrix))[0][0], \
                                                           np.where(distanceMatrix == np.min(distanceMatrix))[1][0])
        if len(correctAssociationMatrix) == 0:
            correctAssociationMatrix = np.array([rThetaMeasurement.loc[measuredCentroidNumber].R,\
                                                        rThetaMeasurement.loc[measuredCentroidNumber].Theta,predictedCentroidNumber])
        else:
            correctAssociationMatrix = np.vstack((correctAssociationMatrix,\
                                                  np.array([rThetaMeasurement.loc[measuredCentroidNumber].R,\
                                                            rThetaMeasurement.loc[measuredCentroidNumber].Theta,predictedCentroidNumber])))

        #fill in with Inf where the centroid has been associated
        distanceMatrix[predictedCentroidNumber, :] = np.Inf
   
    #create associated df
    try:
        correctAssociationMatrix.shape[1]
        associatedDf = pd.DataFrame({'MeasuredRange':correctAssociationMatrix[:,0], 
                            'MeasuredTheta': correctAssociationMatrix[:,1],
                            'CentroidNumber': correctAssociationMatrix[:,2]})
    except:
        associatedDf = pd.DataFrame({'MeasuredRange':correctAssociationMatrix[0], 
                                'MeasuredTheta': correctAssociationMatrix[1],
                                'CentroidNumber': correctAssociationMatrix[2]}, index=range(1))

    #measurement has more centroids than associated
    if associatedDf.shape[0] < rThetaMeasurement.shape[0]:
        print('MORE')
        #add those extra centoids 
        for centroidIndex in range(0,rThetaMeasurement.shape[0]):
            measurement = rThetaMeasurement.loc[centroidIndex]
            if measurement.R not in list(associatedDf['MeasuredRange']):
                measurement = {'MeasuredRange': measurement.R, 'MeasuredTheta':measurement.Theta, 'CentroidNumber': max(associatedDf['CentroidNumber']+1)}
                measurement = pd.DataFrame(data=measurement, index=range(1))
                associatedDf = pd.concat([associatedDf,measurement], axis=0, ignore_index = True)                       

    #measurement has less centroids than predicted - add the predicted value of the unassociated centroid into the associatedDf
    if not((np.isinf(distanceMatrix)).all()):
        print('LESS')
        #add predicted value of unassociated centroid to the dataframe
        np.isfinite(distanceMatrix[:,0])
        for centroidIndex in range(0,xPredDf.shape[0]):
            predicted = xPredDf.loc[centroidIndex]
            if predicted.CentroidNumber not in list(associatedDf['CentroidNumber']):
                predicted = {'MeasuredRange': predicted.Range, 'MeasuredTheta':predicted.Theta, 'CentroidNumber': predicted.CentroidNumber}
                predicted = pd.DataFrame(data=predicted, index=range(1))
                associatedDf = pd.concat([associatedDf,predicted], axis=0, ignore_index = True)    
    return associatedDf, xPredDf

In [6]:
#convert x,y to r,theta
def convertCartesianToPolar(cartesianMeasurementCentroidDf):
    measurementR = np.sqrt(np.square(cartesianMeasurementCentroidDf['X']) + np.square(cartesianMeasurementCentroidDf['Y']))
    measurementTheta = np.arctan(cartesianMeasurementCentroidDf['Y']/cartesianMeasurementCentroidDf['X'])
    polarCentroidDf = pd.DataFrame({'CentroidNumber':cartesianMeasurementCentroidDf['CentroidNumber'], 
                                   'R': measurementR,
                                   'Theta': measurementTheta})
    return polarCentroidDf

### Main Code

In [74]:
#initialize variables
deltaT = 50*10**-3 #50ms

#system matrix
A = np.array([
    [1, deltaT, 0,0],
    [0,1,0,0],
    [0,0,1,deltaT],
    [0,0,0,1]
])
#state transition matrix
F = expm(A*deltaT)
#output matrix
H = np.array([[1,0,0,0],
              [0,0,1,0]])
#covariance matrices
Q = np.eye(4)*0.9
R = np.ones(2).reshape(-1,1)
Pc = np.eye(4)

Pd = np.add(np.matmul(np.matmul(A,Pc), np.transpose(A)), Q) #prediction covariance
S = np.add(R, np.matmul(np.matmul(H, Pd), np.transpose(H))) #innovation covariance
K = np.matmul(Pd, np.matmul(np.transpose(H), np.linalg.inv(S))) #kalman gain
Pupdate = np.subtract(Pd, np.matmul(np.matmul(K, S), np.transpose(K))) #update covariance

In [86]:
#read in measurements from csv
rawCentroidData = pd.read_csv('testCSV.csv', header=None)
#find headers and frames within headers
#each header has the structure X, Y, CentroidNumber
#below each header is the frame data
centroidFramesCartesianMeasurement = list()
headerFound = False
for rowIndex in range(0, rawCentroidData.shape[0]):
    row = rawCentroidData.loc[rowIndex]
    if row[0] == 'X' and row[1] == 'Y' and row[2] == 'CentroidNumber':
        if headerFound: 
            #actual data was found last frame and this frame actual data is found again
            #past frame ended so add to list
            centroidFramesCartesianMeasurement.append(frame)
            frame = pd.DataFrame([])
        else:
            #header found
            headerFound = True
            frame = pd.DataFrame([])
        #data should be following
    elif headerFound:
        if np.isnan(np.float(row[0])) and np.isnan(np.float(row[1])):
            #empty row 
            centroidFramesCartesianMeasurement.append(frame)
            headerFound = False
            #only time its going to be NaN if the frame is completely empty
        else:
            #actual data
            X = np.float(row[0])
            Y = np.float(row[1])
            CentroidNumber = int(row[2])
            if len(frame) == 0:
                frame = pd.DataFrame({'X':X, 'Y':Y, 'CentroidNumber':CentroidNumber}, index=range(1))
            else:
                data = pd.DataFrame({'X':X, 'Y':Y, 'CentroidNumber':CentroidNumber}, index=range(1))
                frame = pd.concat([frame,data])

In [75]:
filteredCentroidPolarFrames = list()

#load in frames
for centroidFrame in centroidFramesCartesianMeasurement:
    filteredPolarFrame = np.array([])
    if len(centroidFrame) > 0:
        rThetaMeasurement = convertCartesianToPolar(centroidFrame)

        if len(filteredCentroidPolarFrames) == 0: #first iteration of the kalman filter
            polarCentroidFrameMeasurement = convertCartesianToPolar(centroidFrame)
            for centroid in polarCentroidFrameMeasurement['CentroidNumber']:
                x = np.expand_dims(np.array([0,0,0,0]), axis=1) #initialise 
                xPred = predictStep(x, A)
                measurement = np.expand_dims(polarCentroidFrameMeasurement.values[int(centroid)][1:], axis=1)
                innovation = innovationStep(xPred, measurement, H)
                xUpdate = UpdateStep(xPred, innovation, K)
                if len(filteredPolarFrame) == 0:
                    filteredPolarFrame = xUpdate
                else:
                    filteredPolarFrame = np.hstack((filteredPolarFrame,xUpdate))
        else:

            associatedDf, xPredDf = dataAssociation(rThetaMeasurement, filteredCentroidPolarFrames)
            overallDf = pd.merge(associatedDf, xPredDf, on='CentroidNumber', how='inner')
            for index in range(0 , overallDf.shape[0]):
                centroidData = overallDf.loc[index]
                measurement = np.expand_dims(np.array([centroidData.MeasuredRange, centroidData.MeasuredTheta]), axis=1)
                xPred = np.expand_dims(np.array([centroidData.PredictedRange, centroidData.PredictedDoppler,centroidData.PredictedTheta, centroidData.PredictedAngularVelocity]), axis=1)
                innovation = innovationStep(xPred, measurement, H)
                xUpdate = UpdateStep(xPred, innovation, K)
                print('UPDATE')
                if len(filteredPolarFrame) == 0:
                    filteredPolarFrame = xUpdate
                else:
                    filteredPolarFrame = np.hstack((filteredPolarFrame,xUpdate))
    elif len(centroidFrame) == 0 and len(filteredCentroidPolarFrames) != 0:
        #all centroids dissapear from frame
        previousCentroids = filteredCentroidPolarFrames[-1]
        xPredAll = np.array([])
        for centroidIndex in list(previousCentroids['CentroidNumber']):
            xPrev = np.expand_dims(previousCentroids.loc[centroidIndex][:-1].values, axis=1)
            xPred = predictStep(xPrev, A)
            if len(xPredAll) == 0:
                xPredAll = xPred
            else:
                xPredAll = np.hstack((xPredAll, xPred))
        xUpdateAll = xPredAll #for understanding purposes
        filteredPolarFrame = xUpdateAll #for understanding purposes
    else:
        continue
        
    filteredPolarDf = pd.DataFrame(np.transpose(filteredPolarFrame))
    filteredPolarDf.columns = ['FilteredRange', 'FilteredDoppler', 'FilteredTheta', 'FilteredAngularVelocity']
    filteredPolarDf['CentroidNumber'] = pd.Series(np.arange(filteredPolarDf.shape[0]))
    filteredCentroidPolarFrames.append(filteredPolarDf)

UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE
UPDATE

In [91]:
#read in measurements from csv
rawCentroidData = pd.read_csv('testCSV.csv', header=None)
#find headers and frames within headers
#each header has the structure X, Y, CentroidNumber
#below each header is the frame data
measurementList = list()
headerFound = False
for rowIndex in range(0, rawCentroidData.shape[0]):
    row = rawCentroidData.loc[rowIndex]
    if row[0] == 'X' and row[1] == 'Y' and row[2] == 'CentroidNumber':
        if headerFound: 
            #actual data was found last frame and this frame actual data is found again
            #past frame ended so add to list
            measurementList.append(frame)
            frame = pd.DataFrame([])
        else:
            #header found
            headerFound = True
            frame = pd.DataFrame([])
        #data should be following
    elif headerFound:
        if np.isnan(np.float(row[0])) and np.isnan(np.float(row[1])):
            #empty row 
            measurementList.append(frame)
            headerFound = False
            #only time its going to be NaN if the frame is completely empty
        else:
            #actual data
            X = np.float(row[0])
            Y = np.float(row[1])
            CentroidNumber = int(row[2])
            if len(frame) == 0:
                frame = pd.DataFrame({'X':X, 'Y':Y, 'CentroidNumber':CentroidNumber}, index=range(1))
            else:
                data = pd.DataFrame({'X':X, 'Y':Y, 'CentroidNumber':CentroidNumber}, index=range(1))
                frame = pd.concat([frame,data])

In [102]:
#plot
xPositions = np.array([])
yPositions = np.array([])
for centroidFrame in filteredCentroidPolarFrames:
    x = np.multiply(centroidFrame.FilteredRange, np.cos(centroidFrame.FilteredTheta))
    y = np.multiply(centroidFrame.FilteredRange, np.sin(centroidFrame.FilteredTheta))
    if len(xPositions) == 0:
        xPositions = x
    else:
        xPositions = np.append(xPositions, x)       
    if len(yPositions) == 0:
        yPositions = y
    else:
        yPositions = np.append(yPositions,y)

s1.setData(xPositions, yPositions)
QtGui.QApplication.processEvents() 

In [73]:
yPositions

array([ 3.28447094e+00,  3.87745356e+00,  3.24898879e+00,  2.53725538e+00,
        2.24498628e+00,  1.93507038e+00,  1.93061881e+00,  1.65444017e+00,
        1.60097148e+00,  1.63439443e+00,  1.56941852e+00,  1.60904882e+00,
        1.52659333e+00,  1.52635372e+00,  1.60222357e+00,  1.62094048e+00,
        1.64257789e+00,  1.64090437e+00,  1.74302710e+00,  1.66938147e+00,
        1.61143941e+00,  1.56218772e+00,  1.51022848e+00,  1.59639763e+00,
        1.61556987e+00,  1.52537889e+00,  1.35627291e+00,  1.30642171e+00,
        1.27219780e+00,  1.51077311e+00,  1.27286289e+00,  1.25286845e+00,
        1.24965295e+00,  1.50409876e+00,  1.49903807e+00,  1.57168600e+00,
        1.25211585e+00,  1.40886376e+00,  1.32236748e+00,  1.51167563e+00,
        1.50679357e+00,  1.50189233e+00,  1.34904128e+00,  1.47957653e+00,
        1.63326565e+00,  1.38696652e+00,  1.29711955e+00,  1.35390556e+00,
        1.39366213e+00,  1.34973000e+00,  1.65866056e+00,  1.67672740e+00,
        1.63987733e+00,  

### Unit Testing

In [8]:
#test predict step
x = np.expand_dims(np.array([1,0.5,0,0]), axis=1) #previous state
xPred = predictStep(x, A) #predicted state
xPred

array([[1.025],
       [0.5  ],
       [0.   ],
       [0.   ]])

In [9]:
#test innovation
#use xPred from previous step
# innovation(xPred, Pd, z, H, R)
z = np.expand_dims(np.array([1,0.5]), axis=1) #create column vector
innovation = innovationStep(xPred, z, H)
innovation

array([[-0.025],
       [ 0.5  ]])

In [10]:
#test update step
xNew = UpdateStep(xPred, innovation, K)
xNew

array([[0.88132417],
       [0.49641259],
       [0.38132417],
       [0.0095212 ]])

In [28]:
#test centroid dataset
#centroidDf_A contains 2 centroids
centroidDfCartesian_A = pd.DataFrame({'CentroidNumber':np.arange(0,2), 
                             'X':np.array([-1,0]), 
                             'Y':np.array([3,1])})
#centroidDf_B contains 2 centroids and continues on from centroidDf_A
centroidDfCartesian_B = pd.DataFrame({'CentroidNumber':np.arange(0,2), 
                             'X':np.array([-1,0.5]), 
                             'Y':np.array([2.5,0.7])})
#centroidDf_C contains 3 centroids continuing from centroidDf_B
centroidDfCartesian_C = pd.DataFrame({'CentroidNumber':np.arange(0,3), 
                             'X':np.array([-1,0.7, 0.3]), 
                             'Y':np.array([1.2,0.5, 3.2])})

centroidDfCartesian_D = pd.DataFrame([])

#contains the centroid frames, where each frame represents a mmWave capture frame
centroidFramesCartesianMeasurement = [centroidDfCartesian_A, centroidDfCartesian_B,centroidDfCartesian_D]

In [23]:
filteredCentroidPolarFrames = list()

#load in frames
for centroidFrame in centroidFramesCartesianMeasurement:
    filteredPolarFrame = np.array([])
    
    if len(filteredCentroidPolarFrames) == 0: #first iteration of the kalman filter
        polarCentroidFrameMeasurement = convertCartesianToPolar(centroidFrame)
        for centroid in polarCentroidFrameMeasurement['CentroidNumber']:
            x = np.expand_dims(np.array([0,0,0,0]), axis=1) #initialise 
            xPred = predictStep(x, A)
            measurement = np.expand_dims(polarCentroidFrameMeasurement.values[centroid][1:], axis=1)
            innovation = innovationStep(xPred, measurement, H)
            xUpdate = UpdateStep(xPred, innovation, K)
            if len(filteredPolarFrame) == 0:
                filteredPolarFrame = xUpdate
            else:
                filteredPolarFrame = np.hstack((filteredPolarFrame,xUpdate))
    else:
        
        associatedDf, xPredDf = dataAssociation(centroidFrame, filteredCentroidPolarFrames)
        overallDf = pd.merge(associatedDf, xPredDf, on='CentroidNumber', how='inner')
        for index in range(0 , overallDf.shape[0]):
            centroidData = overallDf.loc[index]
            measurement = np.expand_dims(np.array([centroidData.MeasuredRange, centroidData.MeasuredTheta]), axis=1)
            xPred = np.expand_dims(np.array([centroidData.PredictedRange, centroidData.PredictedDoppler,centroidData.PredictedTheta, centroidData.PredictedAngularVelocity]), axis=1)
            innovation = innovationStep(xPred, measurement, H)
            xUpdate = UpdateStep(xPred, innovation, K)
            if len(filteredPolarFrame) == 0:
                filteredPolarFrame = xUpdate
            else:
                filteredPolarFrame = np.hstack((filteredPolarFrame,xUpdate))
                
    filteredPolarDf = pd.DataFrame(np.transpose(filteredPolarFrame))
    filteredPolarDf.columns = ['Filtered Range', 'Filtered Doppler', 'Filtered Theta', 'Filtered Angular Velocity']
    filteredPolarDf['CentroidNumber'] = pd.Series(np.arange(filteredPolarDf.shape[0]))
    filteredCentroidPolarFrames.append(filteredPolarDf)

DONE ASSOCIATING


KeyError: 'X'

In [26]:
filteredCentroidPolarFrames[0]

,Filtered Range,Filtered Doppler,Filtered Theta,Filtered Angular Velocity,CentroidNumber
0,2.684268,0.067023,-1.727055,-0.043122,0
1,0.357702,0.008931,0.928499,0.023183,1


In [27]:
filteredCentroidPolarFrames[1]

,Filtered Range,Filtered Doppler,Filtered Theta,Filtered Angular Velocity,CentroidNumber
0,0.729571,0.018205,0.819885,0.020443,0
1,2.556696,0.063754,-1.326176,-0.033059,1


In [486]:
distanceMAtrixTest = distanceMatrix[2,:] = np.inf

In [381]:
measurement

array([3.21403174, 1.47731955, 2.        ])

In [335]:
rThetaMeasurement['CentroidNumber']

0    0
1    1
2    2
Name: CentroidNumber, dtype: int32

In [177]:
rThetaMeasurement

,CentroidNumber,R,Theta
0,0,2.692582,-1.190290
1,1,0.860233,0.950547


In [176]:
correctAssociationMatrix

array([[ 0.86023253,  0.95054684],
       [ 2.6925824 , -1.19028995]])

In [124]:
colIndex

1